In [33]:
import pandas as pd
import json

<strong>Fetch</strong> tomato sensors data from JSON into dataframe

In [ ]:
# Load the JSON data
with open('../private/mock_sample_data.json', 'r') as file:
    data = json.load(file)

# Convert JSON data to DataFrame
df = pd.DataFrame(data)

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Sort DataFrame by date
df = df.sort_values('date')

# print(df)

<p><strong>Functions</strong> for getting daily, weekly, monthly data and analysis</p>

In [ ]:
def moving_window_analysis(df, window_size, column, interval):
    if interval.lower() != 'daily':
        df[f'{column}_{interval}_mean'] = df[column].rolling(window=window_size).mean()
        df[f'{column}_{interval}_std'] = df[column].rolling(window=window_size).std()
        df[f'{column}_{interval}_rate_change'] = df[column].pct_change(periods=window_size)
    return df

def check_temperature_condition(rolling_mean):
    if rolling_mean < 18 or rolling_mean > 25:
        return 'Not Healthy'
    else:
        return 'Healthy'
    
def check_humidity_condition(rolling_mean):
    if rolling_mean < 18 or rolling_mean > 25:
        return 'Not Healthy'
    else:
        return 'Healthy'
    
def check_soil_moisture_condition(rolling_mean):
    if rolling_mean < 18 or rolling_mean > 25:
        return 'Not Healthy'
    else:
        return 'Healthy'

def check_light_condition(rolling_mean):
    if rolling_mean < 18 or rolling_mean > 25:
        return 'Not Healthy'
    else:
        return 'Healthy'
    
# def check_water_requirement_condition(rolling_mean):
#     if rolling_mean < 18 or rolling_mean > 25:
#         return 'Not Healthy'
#     else:
#         return 'Healthy'

def sensor_condition_output(df):
    # Temperature daily, weekly, monthly
    df['temperature_condition'] = df.apply(
        lambda row: {
            'daily': check_temperature_condition(row['temperature']),
            'weekly': check_temperature_condition(row['temperature_weekly_mean']) if row['is_week_end'] else None,
            'monthly': check_temperature_condition(row['temperature_monthly_mean']) if row['is_month_end'] else None,
        },
        axis=1
    )
    # Humidity daily, weekly, monthly
    df['humidity_condition'] = df.apply(
        lambda row: {
            'daily': check_humidity_condition(row['humidity']),
            'weekly': check_humidity_condition(row['humidity_weekly_mean']) if row['is_week_end'] else None,
            'monthly': check_humidity_condition(row['humidity_monthly_mean']) if row['is_month_end'] else None,
        },
        axis=1
    )
    # Soil moisture daily, weekly, monthly
    df['soil_moisture_condition'] = df.apply(
        lambda row: {
            'daily': check_soil_moisture_condition(row['soil_moisture']),
            'weekly': check_soil_moisture_condition(row['soil_moisture_weekly_mean']) if row['is_week_end'] else None,
            'monthly': check_soil_moisture_condition(row['soil_moisture_monthly_mean']) if row['is_month_end'] else None,
        },
        axis=1
    )
    # Light daily, weekly, monthly
    df['light_condition'] = df.apply(
        lambda row: {
            'daily': check_light_condition(row['light']),
            'weekly': check_light_condition(row['light_weekly_mean']) if row['is_week_end'] else None,
            'monthly': check_light_condition(row['light_monthly_mean']) if row['is_month_end'] else None,
        },
        axis=1
    )
    # Water Requirement daily, weekly, monthly
    # df['water_requirement_condition'] = df.apply(
    #     lambda row: {
    #         'daily': check_water_requirement_condition(row['water_requirement']),
    #         'weekly': check_water_requirement_condition(row['water_requirement_weekly_mean']) if row['is_week_end'] else None,
    #         'monthly': check_water_requirement_condition(row['water_requirement_monthly_mean']) if row['is_month_end'] else None,
    #     },
    #     axis=1
    # )
    return df

<p><strong>Apply window analysis</strong> on each sensors data</p>

In [40]:
# Apply moving window analysis on sensors 
# for a daily, weekly, and monthly time frame
df = moving_window_analysis(df, 1, 'temperature', 'daily')
df = moving_window_analysis(df, 7, 'temperature', 'weekly')
df = moving_window_analysis(df, 30, 'temperature', 'monthly')
df = moving_window_analysis(df, 1, 'humidity', 'daily')
df = moving_window_analysis(df, 7, 'humidity', 'weekly')
df = moving_window_analysis(df, 30, 'humidity', 'monthly')
df = moving_window_analysis(df, 1, 'soil_moisture', 'daily')
df = moving_window_analysis(df, 7, 'soil_moisture', 'weekly')
df = moving_window_analysis(df, 30, 'soil_moisture', 'monthly')
df = moving_window_analysis(df, 1, 'light', 'daily')
df = moving_window_analysis(df, 7, 'light', 'weekly')
df = moving_window_analysis(df, 30, 'light', 'monthly')
df = moving_window_analysis(df, 1, 'water_requirement', 'daily')
df = moving_window_analysis(df, 7, 'water_requirement', 'weekly')
df = moving_window_analysis(df, 30, 'water_requirement', 'monthly')

# Mark every nth day per week and month (starts from 0)
df['is_week_end'] = (df.index % 7 == 6)   
df['is_month_end'] = (df.index % 30 == 29) 

# Get sensor condition output in all time frame
df = sensor_condition_output(df)

In [41]:
columns_order = [
    'date',
    'temperature', 'temperature_weekly_mean', 'temperature_weekly_std', 'temperature_weekly_rate_change', 'temperature_monthly_mean', 'temperature_monthly_std', 'temperature_monthly_rate_change', 'temperature_condition',
    'humidity', 'humidity_weekly_mean', 'humidity_weekly_std', 'humidity_weekly_rate_change', 'humidity_monthly_mean', 'humidity_monthly_std', 'humidity_monthly_rate_change', 'humidity_condition',
    'soil_moisture', 'soil_moisture_weekly_mean', 'soil_moisture_weekly_std', 'soil_moisture_weekly_rate_change', 'soil_moisture_monthly_mean', 'soil_moisture_monthly_std', 'soil_moisture_monthly_rate_change', 'soil_moisture_condition',
    'light', 'light_weekly_mean', 'light_weekly_std', 'light_weekly_rate_change', 'light_monthly_mean', 'light_monthly_std', 'light_monthly_rate_change', 'light_condition',
    'water_requirement', 'water_requirement_weekly_mean', 'water_requirement_weekly_std', 'water_requirement_weekly_rate_change', 'water_requirement_monthly_mean', 'water_requirement_monthly_std', 'water_requirement_monthly_rate_change', 'water_requirement_condition',
    'is_week_end', 'is_month_end'   
]

# Rearrange columns order to the desired order
df = df[columns_order]

In [42]:
# Print in text file 
with open('./fixed_stats_log.txt', 'w') as f:
    f.write(df.to_string())